# Assumptions 

> - The question stated that the data is cleaned, so it is assumed that the supplied tables do not have duplicate rows, missing values etc.

> - Per the assignment, the balance in the 'financial_balances' table  reports end of the day balance, so it was assumed that the balance for a given day is already adjusted for all the transactions on the account on the same day. 

# Approach Followed

- Left joined the financial account table  with financial balance table 
- Left joined the merged table with  financical transations table to get the daily transactions for each account
- Calculated the total amount of  transactions  per financial account by summing up the  transcation amount on an account per day
 
- The end of day balance for a given day should be related to the end of the day balance of the previous day or the next day using the following relation. 

                    end of balance (next_day) = end_of_balance(current_day) + net transaction(next day)
                    end_of_balance (previous day) = end_of_balance(current day) - net transaction(current day)
                    
- The above formulas were used to check if the ending balance on a given day is adjusted correctly from the ending balance of the previous day per the transactions reported in the raw data. To do so, observations where both balance and transcation records are available in the raw data were filtered out. The end of balance was then computed using the formulas above. 
        

- It was found that the balance computed using the transcations do not reflect the balance reported in raw data. If the data were consistent (computed balance = reported balance in the raw data), missing data for transactions could have been  linearly interpolated for all dates and then missing balances could be progressively imputed by using the formulas above for backward fill or forward fill. But this observed discrepancy in the raw data suggests  that there is no added advantage of using the available transactions to correctly impute balance data. Also, given the size of the data, this approach is likely to be time consuming and was avoided due to imposed time limit.

- Add rows in the date range of 2017-01-01 to 2018-09-22 for all accounts
- Impute missing balance entires for each account by using linear interpolation on group level
- Aggregate the results on a customer level



# Inconsistencies:
> The relationship between ending balance and transaction for an account is not consistent in the raw data. (explained above)

>  table 2 lists more customers (1556) than table 1 (962). It was be assumed that table 1 lists all the customer from Brex, and table 2 lists additional customers(may be past clients). 
This script only considered customers listed in table 1 and activities (balance,tansactions) of their corresponding financial accounts by left joining other tables.


# Import Packages

In [1]:
import pandas as pd

# Import Raw Data

In [2]:
custom_acc = pd.read_csv( 'customer_accounts.csv' )
fin_acc = pd.read_csv( 'financials_accounts.csv' ) 
fin_bal = pd.read_csv( 'financials_balances.csv',  parse_dates=['accrual_date']) 
fin_trans = pd.read_csv( 'financials_transactions.csv',parse_dates=['accrual_date'])

# Quick Inspection 

## Table 1 : customer_account

In [3]:
custom_acc.head()

,id,name
0,aacc_c10cwqghq00001owfyvwq126,Customer account 1
1,aacc_c8lodc9z00001pjfyvwq126,Customer account 2
2,aacc_czj999vto000j1mo1qkb1ydg,Customer account 3
3,aacc_celx54f0r000a1qqh4lnefn9,Customer account 4
4,aacc_cnq7b2pys000a1t4h4lnefn9,Customer account 5


In [4]:
custom_acc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 962 entries, 0 to 961
Data columns (total 2 columns):
id      962 non-null object
name    962 non-null object
dtypes: object(2)
memory usage: 15.1+ KB


In [5]:
custom_acc.nunique()

id      962
name    962
dtype: int64

## Table 2: financial_account

In [6]:
fin_acc.head()

,id,customer_account_id,name
0,finacc_ctzrz94k00051v354687jg1,aacc_c10cwqghq00001owfyvwq126,Bank account 1
1,finacc_ctzrz8vz00031v3wyouo0dk,aacc_c10cwqghq00001owfyvwq126,Bank account 2
2,finacc_ctzrz91d00041v3h6cklh2s,aacc_c10cwqghq00001owfyvwq126,Bank account 3
3,finacc_ctzrz98x00071v3k3hz7qls,aacc_c10cwqghq00001owfyvwq126,Bank account 4
4,finacc_ctzrz9aw00081v3ghgxf64s,aacc_c10cwqghq00001owfyvwq126,Bank account 5


In [7]:
fin_acc.nunique()

id                     4704
customer_account_id    1556
name                    106
dtype: int64

- Observation: table 2 lists more customers (1556) than table 1 (962)

## Table 3: financial_balance

In [8]:
fin_bal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193500 entries, 0 to 193499
Data columns (total 4 columns):
id              193500 non-null object
account_id      193500 non-null object
amount          193500 non-null int64
accrual_date    193500 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 5.9+ MB


## Table 4: financial_transaction

In [9]:
fin_trans.head()

,id,account_id,amount,accrual_date
0,fintrn_clqswgddo000d1o70bw046m1l,finacc_clqsrmqo800081o70ghgxf64s,-1142,2017-10-04
1,fintrn_clqswgdgq000e1o707aru17jl,finacc_clqsrmqo800081o70ghgxf64s,-1800,2017-10-02
2,fintrn_clqsz0mzq000i1o700nzc0xi4,finacc_clqsrob7p000a1o70h4lnefn9,1043933,2018-03-26
3,fintrn_clqsz0n1b000j1o701qkb1ydg,finacc_clqsrob7p000a1o70h4lnefn9,-1200,2018-03-26
4,fintrn_clqsz0n2z000k1o70zkcqk7ti,finacc_clqsrob7p000a1o70h4lnefn9,-986752,2018-03-26


# Update Column Names for Consistency

In [10]:
# update table 1
custom_acc.rename(columns={"id": "customer_account_id", "name":"customer_name"},inplace = True);
custom_acc.head()

,customer_account_id,customer_name
0,aacc_c10cwqghq00001owfyvwq126,Customer account 1
1,aacc_c8lodc9z00001pjfyvwq126,Customer account 2
2,aacc_czj999vto000j1mo1qkb1ydg,Customer account 3
3,aacc_celx54f0r000a1qqh4lnefn9,Customer account 4
4,aacc_cnq7b2pys000a1t4h4lnefn9,Customer account 5


In [11]:
# update table 2
fin_acc.rename(columns={"id": "finance_account_id", "name":"finance_acc_name"},inplace = True);
fin_acc.head()

,finance_account_id,customer_account_id,finance_acc_name
0,finacc_ctzrz94k00051v354687jg1,aacc_c10cwqghq00001owfyvwq126,Bank account 1
1,finacc_ctzrz8vz00031v3wyouo0dk,aacc_c10cwqghq00001owfyvwq126,Bank account 2
2,finacc_ctzrz91d00041v3h6cklh2s,aacc_c10cwqghq00001owfyvwq126,Bank account 3
3,finacc_ctzrz98x00071v3k3hz7qls,aacc_c10cwqghq00001owfyvwq126,Bank account 4
4,finacc_ctzrz9aw00081v3ghgxf64s,aacc_c10cwqghq00001owfyvwq126,Bank account 5


In [12]:
# Update table 3
fin_bal.rename(columns={"id": " balance_id", "account_id":"finance_account_id","amount" : "balance_amount",
"accrual_date": "date" },inplace = True);
fin_bal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193500 entries, 0 to 193499
Data columns (total 4 columns):
 balance_id           193500 non-null object
finance_account_id    193500 non-null object
balance_amount        193500 non-null int64
date                  193500 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 5.9+ MB


In [13]:
# Update table 4
fin_trans.rename(columns={"id": " trans_id", "account_id":"finance_account_id","amount" : "trans_amount","accrual_date": "date"},inplace = True);
fin_trans.head()

,trans_id,finance_account_id,trans_amount,date
0,fintrn_clqswgddo000d1o70bw046m1l,finacc_clqsrmqo800081o70ghgxf64s,-1142,2017-10-04
1,fintrn_clqswgdgq000e1o707aru17jl,finacc_clqsrmqo800081o70ghgxf64s,-1800,2017-10-02
2,fintrn_clqsz0mzq000i1o700nzc0xi4,finacc_clqsrob7p000a1o70h4lnefn9,1043933,2018-03-26
3,fintrn_clqsz0n1b000j1o701qkb1ydg,finacc_clqsrob7p000a1o70h4lnefn9,-1200,2018-03-26
4,fintrn_clqsz0n2z000k1o70zkcqk7ti,finacc_clqsrob7p000a1o70h4lnefn9,-986752,2018-03-26


# Join Tables to Link Daily Balance and Transactions to Accounts

In [14]:
## Merge Table 1 and Table 2
merged_table_1 = custom_acc.merge(fin_acc, on ='customer_account_id',how ='left' )
merged_table_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2772 entries, 0 to 2771
Data columns (total 4 columns):
customer_account_id    2772 non-null object
customer_name          2772 non-null object
finance_account_id     2721 non-null object
finance_acc_name       2721 non-null object
dtypes: object(4)
memory usage: 108.3+ KB


In [15]:
#left join finance account table with finanace balance table 
merged_table_2 = merged_table_1.merge(fin_bal, on ="finance_account_id",how ='left' )
merged_table_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123250 entries, 0 to 123249
Data columns (total 7 columns):
customer_account_id    123250 non-null object
customer_name          123250 non-null object
finance_account_id     123199 non-null object
finance_acc_name       123199 non-null object
 balance_id            123171 non-null object
balance_amount         123171 non-null float64
date                   123171 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 7.5+ MB


In [16]:
# Remove accounts with no balance info
merged_table_2.dropna(inplace = True)

In [17]:
#merge tables for transaction record
merged_table_3 = merged_table_2.merge(fin_trans, on =["finance_account_id",'date'],how='left' )
merged_table_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259466 entries, 0 to 259465
Data columns (total 9 columns):
customer_account_id    259466 non-null object
customer_name          259466 non-null object
finance_account_id     259466 non-null object
finance_acc_name       259466 non-null object
 balance_id            259466 non-null object
balance_amount         259466 non-null float64
date                   259466 non-null datetime64[ns]
 trans_id              175353 non-null object
trans_amount           175353 non-null float64
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 19.8+ MB


# Verify if balance and transaction records are consistent

In [18]:
# Lets look at dates where both balance and transaction records are available 
merged_table_3[merged_table_3["balance_amount"].notnull()][merged_table_3["trans_amount"].notnull()]
#group records for each finance account and by date
fin_records= merged_table_3.groupby(by=['finance_account_id','date'],as_index=False).sum();
#order data by account and dat
fin_records.sort_values(by=['finance_account_id','date'],inplace=True)

In [19]:
fin_records.head()

,finance_account_id,date,balance_amount,trans_amount
0,finacc_c10t4oek002n1rly31vh9nf,2018-07-16,52408626.0,-1489.0
1,finacc_c10t4oek002n1rly31vh9nf,2018-07-17,26204313.0,0.0
2,finacc_c10t4oek002n1rly31vh9nf,2018-07-18,104655708.0,-40386.0
3,finacc_c10t4oek002n1rly31vh9nf,2018-07-19,26159927.0,-4000.0
4,finacc_c10t4oek002n1rly31vh9nf,2018-07-20,26158745.0,-1182.0


In [20]:
%%time
#define function to compute end of balance
import numpy as np

def calculate_end_balance(balance,trans):
    balance_array = np.asarray(balance)
    trans_array = np.asarray(trans)
    size = len(balance_array)
    
    if size==1:
        return balance # if only one observation, return as is
        
    computed_balance = np.zeros(size)
    #compute using bwd_fill for the first record
    computed_balance[0]=  balance_array[1]-trans_array[1]
    #compute using fwd_fill for the remaining entries
    computed_balance[1:]=  balance_array[:(size-1)]+trans_array[1:size] 
    return computed_balance#(balance_array + trans_array)


Wall time: 0 ns


In [21]:
#compute daily end of balance for all accounts
data=[]
for account in sorted(list(set(fin_records.finance_account_id))):
    df = fin_records[fin_records.finance_account_id==account]
    df.reset_index(inplace=True)
    df['computed_balance'] = calculate_end_balance(df.balance_amount,df.trans_amount)
    data.append(df)
computed_table = pd.concat(data)

C:\Users\shamabar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
computed_table[0:10]

,index,finance_account_id,date,balance_amount,trans_amount,computed_balance
0,0,finacc_c10t4oek002n1rly31vh9nf,2018-07-16,52408626.0,-1489.0,26204313.0
1,1,finacc_c10t4oek002n1rly31vh9nf,2018-07-17,26204313.0,0.0,52408626.0
2,2,finacc_c10t4oek002n1rly31vh9nf,2018-07-18,104655708.0,-40386.0,26163927.0
3,3,finacc_c10t4oek002n1rly31vh9nf,2018-07-19,26159927.0,-4000.0,104651708.0
4,4,finacc_c10t4oek002n1rly31vh9nf,2018-07-20,26158745.0,-1182.0,26158745.0
5,5,finacc_c10t4oek002n1rly31vh9nf,2018-07-23,156911262.0,-6868.0,26151877.0
6,6,finacc_c10t4oek002n1rly31vh9nf,2018-07-24,101872544.0,-683741.0,156227521.0
7,7,finacc_c10t4oek002n1rly31vh9nf,2018-07-25,50926472.0,-4900.0,101867644.0
8,8,finacc_c10t4oek002n1rly31vh9nf,2018-07-27,50471790.0,-4375.0,50922097.0
9,9,finacc_c10t4oek002n1rly31vh9nf,2018-07-30,25235895.0,0.0,50471790.0


The records in the raw data are not consistent. For example, if we calculate the ending balance of 
2018-07-25 based on the transaction on the day and ending balance on the previous day, the explected balance is 101867644. But raw data reports the actual balance to be 50926472.

This means there is no advantage of imputing missing balance entires using transaction data.

# Add rows for missing calendar days

In [23]:
# Create a  dataframe combining all dates between ('1/1/2017', 9/22/2018) and accounts
dates = pd.date_range(start='1/1/2017', end='9/22/2018')
size = len(dates)
result = pd.DataFrame()

for account in list(set(merged_table_3.finance_account_id)):
        data =  {'date':dates,'finance_account_id': [account]*size}
        df = pd.DataFrame(data)
        result= pd.concat([result,df])

# join  dataframe result to merged_table_1
all_data=merged_table_3.merge(result,on=['finance_account_id','date'], how ='right');
all_data.sort_values(by=['finance_account_id','date'],inplace=True)
all_data.shape

(1835886, 9)

In [24]:
all_data.balance_amount.isnull().sum()

1576420

# Impute Missing Values 

In [25]:
#impute by interpolting on a group level
def impute_missing_bal(grp_data):
    res=grp_data.set_index('date').interpolate(method='linear',limit=5).ffill().bfill()
    del res['finance_account_id']
    return res
    
all_data=all_data.groupby(['finance_account_id']).apply(lambda grp_data:impute_missing_bal(grp_data));
all_data.reset_index(inplace=True)
all_data[500:550]

,finance_account_id,date,customer_account_id,customer_name,finance_acc_name,balance_id,balance_amount,trans_id,trans_amount
500,finacc_c10t4oek002n1rly31vh9nf,2018-05-16,cuacc_clitbxcx303bg1vlkky084lt,Customer account 427,Bank account 2,finblce_c11ujh6q002u1rlvkmjn47f,26204313.0,fintrn_c11utxef002v1rly8v3ucsm,-1189.0
501,finacc_c10t4oek002n1rly31vh9nf,2018-05-17,cuacc_clitbxcx303bg1vlkky084lt,Customer account 427,Bank account 2,finblce_c11ujh6q002u1rlvkmjn47f,26204313.0,fintrn_c11utxef002v1rly8v3ucsm,-1189.0
502,finacc_c10t4oek002n1rly31vh9nf,2018-05-18,cuacc_clitbxcx303bg1vlkky084lt,Customer account 427,Bank account 2,finblce_c11ujh6q002u1rlvkmjn47f,26204313.0,fintrn_c11utxef002v1rly8v3ucsm,-1189.0
503,finacc_c10t4oek002n1rly31vh9nf,2018-05-19,cuacc_clitbxcx303bg1vlkky084lt,Customer account 427,Bank account 2,finblce_c11ujh6q002u1rlvkmjn47f,26204313.0,fintrn_c11utxef002v1rly8v3ucsm,-1189.0
504,finacc_c10t4oek002n1rly31vh9nf,2018-05-20,cuacc_clitbxcx303bg1vlkky084lt,Customer account 427,Bank account 2,finblce_c11ujh6q002u1rlvkmjn47f,26204313.0,fintrn_c11utxef002v1rly8v3ucsm,-1189.0
505,finacc_c10t4oek002n1rly31vh9nf,2018-05-21,cuacc_clitbxcx303bg1vlkky084lt,Customer account 427,Bank account 2,finblce_c11ujh6q002u1rlvkmjn47f,26204313.0,fintrn_c11utxef002v1rly8v3ucsm,-1189.0
506,finacc_c10t4oek002n1rly31vh9nf,2018-05-22,cuacc_clitbxcx303bg1vlkky084lt,Customer account 427,Bank account 2,finblce_c11ujh6q002u1rlvkmjn47f,26204313.0,fintrn_c11utxef002v1rly8v3ucsm,-1189.0
507,finacc_c10t4oek002n1rly31vh9nf,2018-05-23,cuacc_clitbxcx303bg1vlkky084lt,Customer account 427,Bank account 2,finblce_c11ujh6q002u1rlvkmjn47f,26204313.0,fintrn_c11utxef002v1rly8v3ucsm,-1189.0
508,finacc_c10t4oek002n1rly31vh9nf,2018-05-24,cuacc_clitbxcx303bg1vlkky084lt,Customer account 427,Bank account 2,finblce_c11ujh6q002u1rlvkmjn47f,26204313.0,fintrn_c11utxef002v1rly8v3ucsm,-1189.0
509,finacc_c10t4oek002n1rly31vh9nf,2018-05-25,cuacc_clitbxcx303bg1vlkky084lt,Customer account 427,Bank account 2,finblce_c11ujh6q002u1rlvkmjn47f,26204313.0,fintrn_c11utxef002v1rly8v3ucsm,-1189.0


# Aggregate on a Customer Level

In [26]:
all_data.drop(['trans_amount'], axis=1,inplace=True)

submission=all_data.groupby(by=['customer_account_id','date'],as_index=False).sum()
submission[600:660]

,customer_account_id,date,balance_amount
600,aacc_c10cwqghq00001owfyvwq126,2018-08-24,5.334731e+09
601,aacc_c10cwqghq00001owfyvwq126,2018-08-25,4.481182e+09
602,aacc_c10cwqghq00001owfyvwq126,2018-08-26,4.481830e+09
603,aacc_c10cwqghq00001owfyvwq126,2018-08-27,5.359600e+09
604,aacc_c10cwqghq00001owfyvwq126,2018-08-28,5.483535e+09
605,aacc_c10cwqghq00001owfyvwq126,2018-08-29,5.442756e+09
606,aacc_c10cwqghq00001owfyvwq126,2018-08-30,6.029608e+09
607,aacc_c10cwqghq00001owfyvwq126,2018-08-31,6.665397e+09
608,aacc_c10cwqghq00001owfyvwq126,2018-09-01,4.432406e+09
609,aacc_c10cwqghq00001owfyvwq126,2018-09-02,4.432406e+09


In [27]:
submission.columns = ['customer_account_id','date','balance']
submission.to_csv('submission.csv', encoding = 'utf-8')